# TASK 01: Basic data operations on DataFrames and RDDs
> Load the dataset movies.json, which is available in the folder Googgle Drive\DemoNov29

In [60]:
import os
import sys
import findspark
findspark.init(os.environ.get('SPARK_HOME'))
PROJECT_HOME = os.path.abspath(os.curdir)
print(PROJECT_HOME)

import pandas as pd
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions \
    import lower, array_contains, size, collect_set, explode, col, count

/home/noobcoder/0_Project/school/BigData/DemoNov29


## SETUP

In [2]:
SPARK_MASTER_HOST = os.environ.get('SPARK_MASTER_HOST', 'localhost')
print (SPARK_MASTER_HOST)
sc = SparkSession.builder\
    .master(f'spark://{SPARK_MASTER_HOST}:7077')\
    .appName('Quiz04')\
    .getOrCreate()
df = sc.read.json(f'file://{PROJECT_HOME}/data/movies.json')

localhost


In [3]:
pd.set_option('display.max_colwidth', 200)

### a. Read the dataset into a DataFrame df and show its schema

In [4]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



### b. Count the number of records in df

In [5]:
df.count()

28795

### c. Count the number of movies that were released after 2015

In [6]:
df.filter(df['year'] > 2015).count()

617

### d. Show the movies whose names contain the word "Soldiers" (case-insensitive)

In [7]:
df.filter(lower(df['title']).contains("soldiers")).toPandas()['title']

0                           Soldiers of Fortune
1                            Soldiers and Women
2                         Soldiers of the Storm
3                              13 Lead Soldiers
4                                Soldiers Three
5                            The Horse Soldiers
6                                  Toy Soldiers
7                                Small Soldiers
8                              Buffalo Soldiers
9                              We Were Soldiers
10                               Paper Soldiers
11    13 Hours: The Secret Soldiers of Benghazi
Name: title, dtype: object

### e. Show the movies that Amy Adams participated

In [8]:
df.filter(array_contains(df['cast'], 'Amy Adams')).toPandas()

,cast,genres,title,year
0,"[Robin Dunne, Sarah Thompson, Keri Lynn Pratt, Amy Adams]","[Comedy, Drama]",Cruel Intentions 2,2001
1,"[Leonardo DiCaprio, Tom Hanks, Christopher Walken, Amy Adams, Martin Sheen, Nathalie Baye, James Brolin]","[Comedy, Drama]",Catch Me If You Can,2002
2,"[Embeth Davidtz, Alessandro Nivola, Scott Wilson, Celia Weston, Amy Adams]",[Drama],Junebug,2005
3,"[Amy Adams, Patrick Dempsey, James Marsden, Susan Sarandon]",[Fantasy],Enchanted,2007
4,"[Zach Braff, Amanda Peet, Jason Bateman, Charles Grodin, Mia Farrow, Donal Logue, Amy Poehler, Amy Adams, Fred Armisen]","[Romance, Comedy]",The Ex,2007
5,"[Alex Neuberger, Amy Adams, Jason Lee]",[Superhero],Underdog,2007
6,"[Meryl Streep, Philip Seymour Hoffman, Amy Adams, Viola Davis]",[Drama],Doubt,2008
7,"[Meryl Streep, Amy Adams, Stanley Tucci, Chris Messina, Linda Emond]","[Biography, Romance, Comedy]",Julie & Julia,2009
8,"[Ben Stiller, Dick Van Dyke, Robin Williams, Amy Adams, Hank Azaria, Owen Wilson, Steve Coogan, Jonah Hill, Ricky Gervais]","[Adventure, Family]",Night at the Museum: Battle of the Smithsonian,2009
9,"[Mark Wahlberg, Christian Bale, Amy Adams, Melissa Leo]","[Drama, Sports]",The Fighter,2010


### f. Check whether Tom Cruise participated in any documentary movies. If no, return false. If yes, return the list of movies (only names enough)

In [9]:
tom_cruise_documentary = df.where(
    array_contains(df['cast'], 'Tom Cruise')\
    & array_contains(df['genres'], 'Documentary')
).toPandas()
if not tom_cruise_documentary.empty:
    tom_cruise_documentary['title']
else:
    print ("Tom Cruise say no with Documentaries")

Tom Cruise say no with Documentaries


### g. Sort the movies following the descending order of the number of actors/actresses participated and then show the top 5 movies

In [10]:
df.sort(size(df['cast']).desc()).toPandas().head(5)

,cast,genres,title,year
0,"[Kristen Stewart, Robert Pattinson, Taylor Lautner, Nikki Reed, Peter Facinelli, Elizabeth Reaser, Ashley Greene, Kellan Lutz, Jackson Rathbone, Julia Jones, Booboo Stewart, Billy Burke, Sarah Cla...","[Fantasy, Romance]",The Twilight Saga: Breaking Dawn - Part 2,2012
1,"[Will Ferrell, Christina Applegate, Paul Rudd, David Koechner, Steve Carell, Kristen Wiig, James Marsden, Dylan Baker, Meagan Good, Harrison Ford, Greg Kinnear, Josh Lawson, Vince Vaughn, Luke Wil...",[Comedy],Anchorman 2: The Legend Continues,2013
2,"[Owen Wilson, Larry the Cable Guy, Michael Caine, Emily Mortimer, Eddie Izzard, Jason Isaacs, Thomas Kretschmann, Joe Mantegna, Peter Jacobson, Tony Shalhoub, Guido Quaroni, Paul Dooley, John Ratz...","[Animated, Family, Spy]",Cars 2,2011
3,"[Neil Patrick Harris, Jayma Mays, George Lopez, (voice), Katy Perry, (voice), Hank Azaria, Sofía Vergara, Alan Cumming, (voice), Jonathan Winters, (voice), Anton Yelchin, (voice), Paul Reubens, (v...","[Family, Live Action]",The Smurfs,2011
4,"[Robert Downey Jr., Chris Hemsworth, Mark Ruffalo, Chris Evans, Scarlett Johansson, Benedict Cumberbatch, Don Cheadle, Tom Holland, Chadwick Boseman, Paul Bettany, Elizabeth Olsen, Anthony Mackie,...","[Superhero, Action, Adventure]",Avengers: Infinity War,2018


### h. List all the genres of movies that Emma Roberts participated

In [19]:
emma_movies = df.where(array_contains(df['cast'], 'Emma Roberts'))\
    .withColumn('genres', explode(df['genres']))
emma_genres = emma_movies.agg(collect_set('genres').alias('d_genres'))
emma_genres.withColumn('d_genres', explode(emma_genres['d_genres'])).show()

+---------+
| d_genres|
+---------+
| Thriller|
|  Fantasy|
|    Crime|
|   Comedy|
|    Drama|
|   Horror|
|  Romance|
|     Teen|
|Biography|
|   Family|
+---------+



### i. Count the number of genres in the dataset

In [25]:
df.withColumn('genres', explode(df['genres']))\
    .agg(collect_set('genres').alias('all_genres'))\
    .withColumn('all_genres', explode(col('all_genres')))\
    .count()

41

### j. Determine the year with highest number of actors/actresses participated

In [119]:
df.withColumn('cast', explode(col('cast')))\
    .groupBy('year')\
    .agg(count('cast').alias('sum_cast'))\
    .orderBy(col('sum_cast').desc())\
    .limit(1)\
    .show()

+----+--------+
|year|sum_cast|
+----+--------+
|2012|    2293|
+----+--------+

